#Analisador de Processo de Compra

Objetivo 1: Construir as mensagens dos emails para envio das notas de empenho

Objetivo 2: Listar os produtos fracassados e checar se todos os produtos homologados pela DG constam nas notas de empenho anexadas ao processo

Objetivo 3: Criar uma Planilha para acompanhamento das entregas dos produtos pelos fornecedores

Objetivo 4: Checar se todos os pagamentos aos fornecedores já foram feitos para poder arquivar o processo

In [61]:
!pip install nltk



In [62]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
!pip install PyPDF2

In [64]:
import PyPDF2

# Abrir o arquivo PDF
with open('/content/drive/MyDrive/Colab Notebooks/Processo_compras/SEI_03699_2024.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)

    # Extrair texto de todas as páginas
    textPDF = ''
    for page in reader.pages:
        textPDF += page.extract_text()

    # Obter o número de páginas
    num_pages = len(reader.pages)

# Exibir o número de páginas
print(f"O documento contém {num_pages} páginas.")

O documento contém 1467 páginas.


O processo é carregado para textPDF

In [65]:
#print(textPDF)

In [66]:
#capturar os dados principais das notas de empenho

import nltk
nltk.download('punkt')
# Dividir o texto em linhas
lines = nltk.tokenize.sent_tokenize(textPDF)


notas_de_empenho = [line for line in lines if "Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL" in line]

# Exibir as linhas correspondentes
for nota_emp in notas_de_empenho:
    print(nota_emp)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1245Data e hora da consulta:
Usuário:  13/06/2024 16:46
        ***.929.604-**
Nota de Empenho
 
 
      Impressão Completa
Código Nome Moeda
Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL - (R$)
2024 NE 332
Natureza da Despesa Esfera UGR PTRES Plano Interno Fonte de RecursoCélula Orçamentária
1 167636 1000000000 339030 70226 ADM MATMOD
Data de Emissão Tipo Processo Taxa de Câmbio Valor
Código Nome
Descrição
Local da Entrega
Informação Complementar04/06/2024 Ordinário 3699/2024 0,0000 9.862,81
54.416.857/0001-16 L & C PARTICIPACOES E COMERCIO LTDA
AQUISIÇÃO DE MATERIAL FARMACOLÓGICO, HOSPITALAR E ODONTOLÓGICO PARA A SAMS - PREGÃO
ELETRÔNICO Nº 90023/2024 - PROCESSO SEI Nº 3699/2024.
1250Data e hora da consulta:
Usuário:  13/06/2024 16:50
        ***.929.604-**
Nota de Empenho
 
 
      Impressão Completa
Código Nome Moeda
Ano Tipo Número70008 TRIBUNAL REGIONAL ELEITORAL DO RIO G.DO NORTE REAL - (R$)
2024 NE 333
Natureza da Despesa Esfera UGR PTRES Plano Interno

In [67]:
len(notas_de_empenho)

11

In [68]:
import pprint
#text = notas_de_empenho[0]

# Criar o dicionário
lista_dict = []
data_dict = {}

for nota_emp in notas_de_empenho:
  # Extrair as informações
  lines = nota_emp.splitlines()

  # Criar o dicionário
  data_dict = {}

  # Extrair as informações
  lines = nota_emp.splitlines()

  # Encontrar e adicionar 'NE'
  for line in lines:
      if "NE" in line:
          parts = line.split()
          data_dict['NE'] = parts[parts.index('NE') + 1]

  # Encontrar e adicionar 'Empresa' e 'CNPJ'
  for line in lines:
      if "/0001-" in line:
          parts = line.split()
          data_dict['CNPJ'] = parts[0]
          data_dict['Empresa'] = ' '.join(parts[1:])
          break  # Para evitar múltiplas capturas

  # Encontrar e adicionar 'Valor' usando referências
  for line in lines:
      if "0,0000" in line:
          # Capturar o valor que vem depois de "0,0000 "
          valor = line.split("0,0000 ")[1].split("\n")[0].strip()
          data_dict['Valor'] = valor
          break  # Para evitar múltiplas capturas

  # Adicionar o dicionário à lista
  lista_dict.append(data_dict)
# Exibir o dicionário
pprint.pprint(lista_dict)

[{'CNPJ': '54.416.857/0001-16',
  'Empresa': 'L & C PARTICIPACOES E COMERCIO LTDA',
  'NE': '332',
  'Valor': '9.862,81'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '333',
  'Valor': '311,14'},
 {'CNPJ': '22.077.847/0001-07',
  'Empresa': 'JOSE DANTAS DINIZ FILHO',
  'NE': '334',
  'Valor': '381,75'},
 {'CNPJ': '19.367.167/0001-13',
  'Empresa': 'COMERCIAL TARGET DE EPI E EPC LTDA',
  'NE': '335',
  'Valor': '1.592,50'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '336',
  'Valor': '6.491,60'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '326',
  'Valor': '126,00'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '348',
  'Valor': '380,00'},
 {'CNPJ': '30.843.402/0001-19',
  'Empresa': 'DOIS K COMERCIO E SERVICOS LTDA',
  'NE': '349',
  'Valor': '145,50'},
 {'CNPJ': '43.492.384/0001-10',
  'Empresa': 'ANA C REGIS',
  'NE': '350',
  'Valor': '515,80'},
 {

In [69]:
#criação do DataFrame df_Notas_de_empenho com os dados do dicionario lista_dict

import pandas as pd
df_Notas_de_empenho = pd.DataFrame(lista_dict)
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81"
1,333,43.492.384/0001-10,ANA C REGIS,"311,14"
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75"
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50"
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60"
5,326,43.492.384/0001-10,ANA C REGIS,"126,00"
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00"
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50"
8,350,43.492.384/0001-10,ANA C REGIS,"515,80"
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95"


In [70]:
# prompt: capturar os emails que existirem no processo

import re

# Expressão regular para encontrar emails
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

# Encontrar todos os emails no texto
emails = re.findall(email_pattern, textPDF)
emails = list(set(emails))

# Remover os emails que forem do 'tre-rn'
filtered_emails = [item for item in emails if 'tre-rn' not in item]

email_minusculas = []
for string in filtered_emails:
    email_minusculas.append(string.lower())

email_minusculas = list(set(email_minusculas))
# Exibir os emails encontrados
pprint.pprint(email_minusculas)


['dantasdiniz@gmail.com',
 'nacregis.parnamirim@gmail.com',
 'lcparticipacoescomercio@gmail.com',
 'doiskcomercio@gmail.com',
 'licitacoes@targethospitalar.com.br',
 'acoescomercio@gmail.com',
 'anacregis.parnamirim@gmail.com']


In [71]:
#pegar os emails

import re

emails = []

for i in range (df_Notas_de_empenho.shape[0]):
  emails.append("nao_encontrado")

for i in range (df_Notas_de_empenho.shape[0]):
  # Texto extraído (exemplo)
  extracted_text = textPDF

  # String a ser encontrada
  company_name = df_Notas_de_empenho['Empresa'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

  # Encontrar a posição da frase que limita a busca
  stop_index = extracted_text.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = extracted_text[:stop_index]

      # Encontrar a posição da empresa no texto
      company_match_name = re.search(re.escape(company_name), text_before_stop)
      #company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_name:
          # Obter o texto após a empresa
          text_after_company_name = text_before_stop[company_match_name.end():]

          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_name)

          if email_match:
              emails[i] = email_match.group()

  else:
      print("Frase de parada não encontrada no texto.")

for i in range (df_Notas_de_empenho.shape[0]):
  # Texto extraído (exemplo)
  extracted_text = textPDF

  # String a ser encontrada
  company_cnpj = df_Notas_de_empenho['CNPJ'][i]
  stop_phrase = 'Solicito análise das propostas'

  # Regex para encontrar um email
  email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

  # Encontrar a posição da frase que limita a busca
  stop_index = extracted_text.find(stop_phrase)

  if stop_index != -1:
      # Obter o texto antes da frase de parada
      text_before_stop = extracted_text[:stop_index]

      # Encontrar a posição da empresa no texto
      company_match_cnpj = re.search(re.escape(company_cnpj), text_before_stop)

      if company_match_cnpj:
          # Obter o texto após a empresa
          text_after_company_cnpj = text_before_stop[company_match_cnpj.end():]

          # Procurar o primeiro email no texto subsequente
          email_match = re.search(email_pattern, text_after_company_cnpj)

          if email_match:
            if emails[i] == "nao_encontrado":
              emails[i] = email_match.group()


  else:
      print("Frase de parada não encontrada no texto.")

df_Notas_de_empenho['Email'] = emails



In [72]:
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",nao_encontrado
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",nacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",nacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [73]:
# gerar lista com empresas cujo email não foi encontrado

lista_empresas_sem_email = []
indices_sem_email = []

for i in range(df_Notas_de_empenho.shape[0]):
  if df_Notas_de_empenho['Email'][i] == "nao_encontrado":
    lista_empresas_sem_email.append(df_Notas_de_empenho['Empresa'][i])
    indices_sem_email.append(i)

# print(lista_empresas_sem_email)
# print(indices_sem_email)

#procurar os emails não encontrados
emails_encontrados = []
for empresa in lista_empresas_sem_email:
  empresa = empresa.split()
  for palavra in empresa:
        for string in email_minusculas:
            if palavra.lower() in string.lower():
                emails_encontrados.append(string)

emails_encontrados = list(set(emails_encontrados))
#print(emails_encontrados)

diferenca = len(lista_empresas_sem_email) - len(emails_encontrados)
#print(diferenca)

while (diferenca >0):
  emails_encontrados.append("nao_encontrado")
  diferenca -= 1

# print(emails_encontrados)

for i in range(len(indices_sem_email)):
    df_Notas_de_empenho['Email'][indices_sem_email[i]] = emails_encontrados[i]


df_Notas_de_empenho

<ipython-input-73-228e223e985d>:36: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Email'][indices_sem_email[i]] = emails_encontrados[i]
<ipython-input-73-228e223e985d>:36: FutureWarning: ChainedAssignmentError: behaviour

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",nacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",nacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [74]:

# Função para encontrar a linha do email no texto extraído
def find_email_line(email):
    # Regex para encontrar o email no texto extraído
    match = re.search(re.escape(email), extracted_text)
    if match:
        # Obter a linha onde o email foi encontrado
        lines = extracted_text.splitlines()
        for line in lines:
            if email in line:
                return line  # Retorna a linha correspondente
    return None  # Retorna None se o email não for encontrado

# Criar nova coluna 'Verif_email' com as linhas correspondentes
df_Notas_de_empenho['Verif_email'] = df_Notas_de_empenho['Email'].apply(find_email_line)

# Exibir o DataFrame atualizado

df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email,Verif_email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"LCPARTICIPACOESCOMERCIO@GMAIL.COM , TELEFONE: ..."
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,Diniz Dantas <dantasdiniz@gmail.com> 14 de jun...
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,(11) 5461-1551 (11) 95146-8131 e-mail: licitac...
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"LCPARTICIPACOESCOMERCIO@GMAIL.COM , TELEFONE: ..."


In [75]:
lista_empresas_sem_email = []
cnjp_empresas_sem_email = []
for i in range (df_Notas_de_empenho.shape[0]):
  if df_Notas_de_empenho['Email'][i] == "nao_encontrado":
    lista_empresas_sem_email.append(df_Notas_de_empenho['Empresa'][i])
    cnpj_limpo = df_Notas_de_empenho['CNPJ'][i].replace('.', '').replace('/', '').replace('-', '')
    cnjp_empresas_sem_email.append(cnpj_limpo)
    df_Notas_de_empenho['Verif_email'][i] = f'https://cnpj.biz/{cnpj_limpo}'

print("Os emails das seguintes empresas não foram encontrados:")
print(lista_empresas_sem_email)
print()
print("Na coluna 'Verif_email' do Dataframe foi gerado um link onde é possível encontrar o email")
df_Notas_de_empenho

Os emails das seguintes empresas não foram encontrados:
['INFINITI EMPREENDIMENTOS LTDA']

Na coluna 'Verif_email' do Dataframe foi gerado um link onde é possível encontrar o email


<ipython-input-75-b16bea498299>:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Verif_email'][i] = f'https://cnpj.biz/{cnpj_limpo}'


,NE,CNPJ,Empresa,Valor,Email,Verif_email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"LCPARTICIPACOESCOMERCIO@GMAIL.COM , TELEFONE: ..."
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,Diniz Dantas <dantasdiniz@gmail.com> 14 de jun...
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,(11) 5461-1551 (11) 95146-8131 e-mail: licitac...
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,E-mail: doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",nacregis.parnamirim@gmail.com,E-mail: a nacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"LCPARTICIPACOESCOMERCIO@GMAIL.COM , TELEFONE: ..."


In [76]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Antes de continuar, cheque o email na coluna do Email do DataFrame com o que aparece na coluna Verif_email no mesmo DataFrame!</h1>'
text2 = '<h1 style="color: red;">Caso seja necessário insira manualmente no DataFrame os emails que o programa não conseguiu detectar ou que esteja escrito errado.</h1>'
text3 = '<h1 style="color: red;">Foi colocado na célula abaixo um print errado para o código parar de executar.</h1>'
# Exibir o texto
display(HTML(text3))
display(HTML(text1))
display(HTML(text2))

In [77]:
pergunta = input("Precisa corrigir algum email? (s/n): ")
if pergunta.lower() == 's':
  print(parar)

Precisa corrigir algum email? (s/n): n


In [78]:
df_Notas_de_empenho['Email'][1] = "anacregis.parnamirim@gmail.com" #email sendo corrigido manualmente
df_Notas_de_empenho['Email'][5] = "anacregis.parnamirim@gmail.com" #email sendo corrigido manualmente
df_Notas_de_empenho['Email'][8] = "anacregis.parnamirim@gmail.com" #email sendo corrigido manualmente
df_Notas_de_empenho['Email'][10] = "infiniti.licitacoes@gmail.com" #email sendo corrigido manualmente
dropar = ['Verif_email']
df_Notas_de_empenho = df_Notas_de_empenho.drop(dropar, axis=1)
df_Notas_de_empenho

<ipython-input-78-ae6d1649ecd1>:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Email'][1] = "anacregis.parnamirim@gmail.com" #email sendo corrigido manualmente
<ipython-input-78-ae6d1649ecd1>:2: FutureWarning: ChainedA

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [79]:

empresas = df_Notas_de_empenho['Empresa'].unique()  # Obter valores únicos da coluna "Empresa"

# Criar um dicionário com listas vazias, onde as chaves são os nomes das empresas
listas_empresas = {empresa: [] for empresa in empresas}

# Iterar sobre o DataFrame e preencher as listas com os números da coluna "NE"
for _, row in df_Notas_de_empenho.iterrows():
    empresa = row['Empresa']
    ne = row['NE']
    listas_empresas[empresa].append(ne)

# Exibir o dicionário preenchido
print(listas_empresas)



{'L & C PARTICIPACOES E COMERCIO LTDA': ['332', '351'], 'ANA C REGIS': ['333', '326', '350'], 'JOSE DANTAS DINIZ FILHO': ['334'], 'COMERCIAL TARGET DE EPI E EPC LTDA': ['335'], 'DOIS K COMERCIO E SERVICOS LTDA': ['336', '348', '349'], 'INFINITI EMPREENDIMENTOS LTDA': ['352']}


In [80]:
from datetime import datetime

# Obter a data atual
current_date = datetime.now()

# Formatar o dia, mês e ano como strings
formatted_date = f"SAMS - TRE-RN, {current_date.day}/{current_date.month}/{current_date.year}."

# Função para criar e-mails
def criar_emails(empresas):
    emails = []
    for empresa, numeros in empresas.items():
        for numero in numeros:

            num_NE = f'{current_date.year}NE000{numero}'
            email = f"""Prezados(as) Senhores(as),

                        Encaminhamos a Nota de Empenho número {num_NE} (em anexo), à empresa {empresa}.
                        Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
                        desta Nota de Empenho.
                        Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
                        A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
                        Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
                        5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
                        FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

                        Atenciosamente,
                        Flávio Roberto Guerra Seabra
                        {formatted_date}
                      """
            emails.append(email)
    return emails

# Chamar a função e obter os e-mails
emails_gerados = criar_emails(listas_empresas)

# Nome do arquivo de saída
output_file = 'textos_emails.txt'

# Abrir o arquivo para escrita
with open(output_file, 'w', encoding='utf-8') as file:

# Exibir os e-mails gerados
    for email in emails_gerados:
        print(email)
        file.write(email)
        print("-" * 40)
        file.write("-" * 40)
        file.write("\n")

Prezados(as) Senhores(as),

                        Encaminhamos a Nota de Empenho número 2024NE000332 (em anexo), à empresa L & C PARTICIPACOES E COMERCIO LTDA.
                        Informamos que o prazo para entrega do material é de até 30 (trinta) dias corridos, contados a partir da data de envio
                        desta Nota de Empenho.
                        Solicitamos especial atenção quanto ao PRAZO, LOCAL e HORÁRIO para entrega do material.
                        A entrega deverá ser realizada na Seção de Assistência Médica e Saúde Ocupacional (SAMS), localizada na Sede
                        Administrativa do TRE/RN. Endereço: Av. Rui Barbosa, 215. Tirol, Natal-RN. CEP: 59.015-290. Telefone: (84) 3654-
                        5399, nos horários de segunda a quinta-feira, das 13 às 18 horas ou na sexta-feira, das 8 às 13 horas.
                        FAVOR CONFIRMAR RECEBIMENTO DESTE E-MAIL enviando resposta para o email sams@tre-rn.jus.br

                       

In [81]:
df_Notas_de_empenho

,NE,CNPJ,Empresa,Valor,Email
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM


In [82]:
from IPython.display import display, HTML

# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Neste ponto o Objetivo 1 deve ter sido obtido</h1>'
text2 = '<h1 style="color: red;">Verifique se o arquivo "textos_emails.txt" foi criado .</h1>'

# Exibir o texto
display(HTML(text1))
display(HTML(text2))

Início de captura dos números dos itens Desertos e Fracassados

In [83]:
import re

# Texto extraído (exemplo)
extracted_text = textPDF

# Definindo o padrão para capturar o trecho desejado
pattern = r'(Pregão eletrônico.*?os demais itens lograram êxito e aguardam adjudicação)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, extracted_text, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    parecer_ajdg_homolog = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [84]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(parecer_ajdg_homolog)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
parecer_ajdg_homolog_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
parecer_ajdg_homolog_tokenizado = [s for s in parecer_ajdg_homolog_tokenizado if s != ',']


In [86]:
# Lista para armazenar os números dos itens desertos
lista_desertos = []

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'item':
      parecer_ajdg_homolog_tokenizado = 'itens'

capturar = False

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'itens' and parecer_ajdg_homolog_tokenizado[i+1].isdigit() and len(lista_desertos) == 0:
      capturar = True

    if parecer_ajdg_homolog_tokenizado[i].isdigit() and capturar == True:
      lista_desertos.append(parecer_ajdg_homolog_tokenizado[i])
    else:
      if len(lista_desertos) > 0:
        capturar = False

print(lista_desertos)

['22', '35', '36', '38', '40', '41', '42', '44', '45', '47', '49', '50', '52', '53', '60', '61', '63', '66', '76', '77', '83', '85', '87']


In [87]:
# Lista para armazenar os números dos itens desertos
lista_fracassados = []

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if parecer_ajdg_homolog_tokenizado[i] == 'item':
      parecer_ajdg_homolog_tokenizado = 'itens'

capturar = False

for i in range(len(parecer_ajdg_homolog_tokenizado)):
    if (parecer_ajdg_homolog_tokenizado[i] == 'desertos' or parecer_ajdg_homolog_tokenizado[i] == 'deserto'):
      capturar = True

    if parecer_ajdg_homolog_tokenizado[i].isdigit() and capturar == True:
      lista_fracassados.append(parecer_ajdg_homolog_tokenizado[i])


print(lista_fracassados)

['20', '74', '84']


Final de captura dos números dos itens Desertos e Fracassados

listas: **lista_fracassados** e **lista_desertos**

Iniciar capturar todos os numeros dos itens do termo de referencia

In [88]:
import re

# Texto extraído (exemplo)
extracted_text = textPDF

# Definindo o padrão para capturar o trecho desejado
pattern = r'(Definição do objeto.*?O objeto desta contratação não se enquadra como sendo de bem de luxo)'

# Encontrar o trecho entre os padrões
match = re.search(pattern, extracted_text, re.DOTALL)

if match:
    # Obter o texto completo entre os padrões
    tabela_produtos = match.group(1).strip()  # O texto entre os dois padrões
else:
    print("Trecho não encontrado.")

In [89]:
lista_textos_para_remover = [ "Câmara Nacional de Modelos de Licitações e Contratos da Consultoria-Geral da União",
                              "Aprovado pela Secretaria de Gestão e Inovação",
                              "UASG 70008"
                            ]

for texto in lista_textos_para_remover:
  tabela_produtos = tabela_produtos.replace(texto, "")

for i in range(1, num_pages + 1):
  tabela_produtos = tabela_produtos.replace(f"/ pg. {i}", "")

In [90]:
print(tabela_produtos)

Definição do objeto
1.1.      Aquisição de material hospitalar e odontológico, nos termos da tabela abaixo, conforme
condições e exigências estabelecidas neste instrumento.
ITEM Especificações CATMAT Unidade de Medida Quant.
1Absorvente higiênico, tamanho normal, 
sem abas, cobertura 100%
algodão. 230246Embalagem com 
8 unidades 10
2Álcool hidratado 70%
líquido, uso Farmacológico. 269941 Litro 10
Bolsa térmica gel, que
molda-se perfeitamente ao
corpo e não congela,
mesmo quando man tida
constantemente no
freezer, reu tilizável, com
gel térmico não tóxico,
para compressas quente e
gelada, podendo ser
Termo de referência (0032434)         SEI 03699/2024  Termo de Referência 5/2024

Modelo de Aquisições — Atualização: maio/2023
2 de 313resfriada no freezer,
aquecida no micro-ondas
ou na água quente.
Tamanho grande
(Comprimento: 54 cm.
Altura: 25 cm). 433757 Unidade 5
4Bolsa térmica gel, que
molda-se perfeitamente ao
corpo e não congela,
mesmo quando man tida
constantemente no
freezer, reu

In [91]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(tabela_produtos)

# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
tabela_produtos_tokenizada = [word for word in tokens if word.lower() not in stop_words]
tabela_produtos_tokenizada = [item for item in tabela_produtos_tokenizada if 'UASG' not in item]

# tabela_produtos_tokenizada

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
#Obter o número do último produto da lista

filtered_list = [s for s in tabela_produtos_tokenizada if re.match(r'^[1-9][0-9A-Z][a-zA-Z]', s)]
filtered_list = [s for s in filtered_list if not (len(s) <= 5 or 'UASG' in s)]
maior = max(filtered_list)
ultimo_produto = ''.join(filter(str.isdigit, maior))
ultimo_produto = int(ultimo_produto)
print(ultimo_produto)


87


In [93]:
# Criar a nova lista com valores de 1 a ultimo_produto, excluindo os números da lista de exclusão
lista_desertos = [int(s) for s in lista_desertos]
lista_fracassados = [int(s) for s in lista_fracassados]

# Criar lista com os produtos que não foram homologados
lista_remover = lista_desertos + lista_fracassados

# Clista com os números dos produtos homologados
produtos_homologados = [i for i in range(1, int(ultimo_produto) + 1) if i not in lista_remover] # Changed ultimo_produto to int(ultimo_produto)
print(produtos_homologados)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 39, 43, 46, 48, 51, 54, 55, 56, 57, 58, 59, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 75, 78, 79, 80, 81, 82, 86]


Obtida a lista de produtos homologados: **produtos_homologados**

Iniciar a checagem se os produtos homologados estão nas notas de empenho

In [94]:
numeros_NE = []
for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = df_Notas_de_empenho['NE'][i]
  integer_value = int(num_nota_de_empenho)
  numeros_NE.append(integer_value)
  numeros_NE = sorted(numeros_NE)

print(numeros_NE)

[326, 332, 333, 334, 335, 336, 348, 349, 350, 351, 352]


In [95]:
#capturar as notas de empenho para a lista notas_de_empenho_completas

import re

# Texto extraído (exemplo)
extracted_text = textPDF
notas_de_empenho_completas = []

for i in range (df_Notas_de_empenho.shape[0]):
  num_nota_de_empenho = numeros_NE[i]
  # Definindo o padrão para capturar o trecho desejado
  pattern = rf'(2024 NE {num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO|2024 NE {num_nota_de_empenho}.*?ANA ESMERA PIMENTEL DA FONSECA)'
  #pattern = rf'(2024 NE.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'
  # pattern = rf'({num_nota_de_empenho}.*?SIMONE MARIA DE OLIVEIRA SOARES MELLO)'

  # Encontrar o trecho entre os padrões
  match = re.search(pattern, extracted_text, re.DOTALL)

  if match:
      # Obter o texto completo entre os padrões
      nota_empenho = match.group(1).strip()  # O texto entre os dois padrões
      notas_de_empenho_completas.append(nota_empenho)
  else:
      print(f"Trecho não encontrado para a NE {num_nota_de_empenho}")



In [96]:
notas_de_empenho_completas

['2024 NE 326\nNatureza da Despesa Esfera UGR PTRES Plano Interno Fonte de RecursoCélula Orçamentária\n1 167636 1000000000 339030 70226 ADM MATAUX\nData de Emissão Tipo Processo Taxa de Câmbio Valor\nCódigo Nome\nDescrição\nLocal da Entrega\nInformação Complementar03/06/2024 Ordinário 3699/2024 0,0000 126,00\n43.492.384/0001-10 ANA C REGIS\nAQUISIÇÃO DE MATERIAL DE CONSUMO PARA A SAMS - PREGÃO ELETRÔNICO Nº 90023/2024 - PROCESSO\nSEI Nº 3699/2024.\nTRE/RN\n07000805900232024 - UASG Minuta: 070008ArtigoCódigo\nParágrafoModalidade de Licitação\nIncíso Ato NormativoAmparo Legal\n179 PREGAO\n28 - I -Alínea\nLei 14.133/2021\nSistema de Origem\nCOMPRASNET-MEFavorecido\nEndereço\nDOUTOR SADI MENDES 1183 SANTOS REISCEP\n59141-085\nMunicípio\nPARNAMIRIM RNUF TelefoneCNPJ\n05.792.645/0001-28CEP\n59015-290Endereço\nAVENIDA RUI BARBOSA, 215         TIROL, NATAL -RN\nMunicípio\nNATALUF\nRNTelefone\n(84)3654-6000UG Emitente\n1 de      2 002Versão Data/Hora\n13/06/2024 17:36:15Operação\nAlteração\nEmp

In [97]:
#inserir as notas de empenho tokenizadas e sem stopwords no DataFrame

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df_Notas_de_empenho['NE_Completa'] = ""

for i in range(len(notas_de_empenho_completas)):
  nltk.download('punkt')
  nltk.download('stopwords')

  # Supondo que 'text' é o conteúdo extraído do PDF
  tokens = word_tokenize(notas_de_empenho_completas[i])

  # Remover stopwords (palavras comuns que não têm significado relevante)
  stop_words = set(stopwords.words('portuguese'))
  df_Notas_de_empenho['NE_Completa'][i] = [word for word in tokens if word.lower() not in stop_words]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-97-6d0376449841>:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [119]:
#Capturar os números dos itens de cada nota de empenho e inserir no DataFrame

import numpy as np

df_Notas_de_empenho['itens_da_NE'] = ''

for i in range(df_Notas_de_empenho.shape[0]):
  num_NE = df_Notas_de_empenho['NE_Completa'][i][2]
  produtos = []
  for j in range(len(df_Notas_de_empenho['NE_Completa'][i])):
    if (df_Notas_de_empenho['NE_Completa'][i][j] == 'Item') and (df_Notas_de_empenho['NE_Completa'][i][j+1] == 'compra'):
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      df_Notas_de_empenho.loc[df_Notas_de_empenho['NE'] == num_NE, 'itens_da_NE'] += ', '
      # df_Notas_de_empenho['itens_da_NE'][i] += df_Notas_de_empenho['NE_Completa'][i][j+3]
      # df_Notas_de_empenho['itens_da_NE'][i] += ', '

# # Remove the last comma and space
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.rstrip(', ')

# # Split the string into a list of strings
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].str.split(', ')

# # Convert the list of strings to a list of integers
df_Notas_de_empenho['itens_da_NE'] = df_Notas_de_empenho['itens_da_NE'].apply(lambda x: [int(i) for i in x])


In [134]:
for i in range(df_Notas_de_empenho.shape[0]):
  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))

<ipython-input-134-6d94fdba1bab>:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['itens_da_NE'][i] = list(set(df_Notas_de_empenho['itens_da_NE'][i]))
<ipython-input-134-6d94fdba1bab>:2: FutureWarning: ChainedAssignmentEr

In [135]:
for i in range(df_Notas_de_empenho.shape[0]):
  print("----")
  print(f"Nota: {df_Notas_de_empenho['NE'][i]}")
  print(f"Produtos: {df_Notas_de_empenho['itens_da_NE'][i]}")

----
Nota: 332
Produtos: [12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 28, 32, 37, 39, 43, 51, 58, 69, 75]
----
Nota: 333
Produtos: [16, 2]
----
Nota: 334
Produtos: [26]
----
Nota: 335
Produtos: [46]
----
Nota: 336
Produtos: [3, 4, 5, 6, 7, 8, 11, 25, 27, 29, 33, 34, 48, 59, 62, 64, 65, 67, 68, 70, 71, 72, 73, 80, 81, 82, 86]
----
Nota: 326
Produtos: [78]
----
Nota: 348
Produtos: [1, 79, 30, 31]
----
Nota: 349
Produtos: [9, 10, 57]
----
Nota: 350
Produtos: [54]
----
Nota: 351
Produtos: [55]
----
Nota: 352
Produtos: [56]


In [136]:
#Criar uma lista com os produtos que foram homologados mas não estão em nenhuma Nota de Empenho

produtos_homologados_nao_presentes_nas_NE = produtos_homologados.copy()

for lista in df_Notas_de_empenho['itens_da_NE']:
  for num in lista:
    if num in produtos_homologados_nao_presentes_nas_NE:
      produtos_homologados_nao_presentes_nas_NE.remove(num)

print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')

Produtos que foram homologados mas não estão em nenhuma NE: 
[]


In [137]:
from IPython.display import display, HTML
text1 = '<h1 style="color: red;">Aparentemente os seguintes produtos que foram homologados não estão em nenhuma Nota de Empenho</h1>'
text2 = '<h1 style="color: red;">FAVOR CHECAR!!!</h1>'
text3 = '<h1 style="color: red;">Todos os produtos homologados foram localizados em alguma Nota de Empenho</h1>'

if (len(produtos_homologados_nao_presentes_nas_NE) >0):
  # Texto em vermelho e com fonte maior
  display(HTML(text1))
  display(HTML(text2))
  print()
  print(f'Produtos que foram homologados mas não estão em nenhuma NE: \n{produtos_homologados_nao_presentes_nas_NE}')
else:
  # Texto em vermelho e com fonte maior
  display(HTML(text3))

In [138]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 2 atingido</h1>'

# Exibir o texto
display(HTML(text1))

#Início de criação da planilha de acompanhamento das entregas (Objetivo 3)

In [139]:
#Cria df_para_planilha

from datetime import datetime, timedelta

# Obter a data atual
current_date = datetime.now()

# Adicionar 30 dias à data atual
new_date = current_date + timedelta(days=30)

formatted_date = new_date.strftime("%d/%m/%Y")

resposta = input("Deseja criar a Planilha de Controle de Entregas? (s/n): ")

if resposta.lower() == 's':
  colunas_dropar = ['NE_Completa', 'itens_da_NE']
  df_para_planilha = df_Notas_de_empenho.drop(colunas_dropar, axis=1)
  df_para_planilha['Fim do prazo para entrega'] = formatted_date
  df_para_planilha['Data da Entrega'] = ''
  df_para_planilha['No. da Nota Fiscal'] = ''
  df_para_planilha['Pendências na Entrega'] = ''
  df_para_planilha['Enviado para pagamento em:'] = ''
  df_para_planilha['Pago em:'] = ''
  df_para_planilha.to_excel('Planilha de Controle de Entregas.xlsx', index=False)

Deseja criar a Planilha de Controle de Entregas? (s/n): s


In [44]:
df_para_planilha

,NE,CNPJ,Empresa,Valor,Email,Fim do prazo para entrega,Data da Entrega,No. da Nota Fiscal,Pendências na Entrega,Enviado para pagamento em:,Pago em:
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,03/11/2024,,,,,
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com,03/11/2024,,,,,
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,03/11/2024,,,,,
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,03/11/2024,,,,,
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,03/11/2024,,,,,
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com,03/11/2024,,,,,
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,03/11/2024,,,,,
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,03/11/2024,,,,,
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com,03/11/2024,,,,,
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,03/11/2024,,,,,


In [140]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 3 atingido</h1>'

# Exibir o texto
display(HTML(text1))

#Início da checagem de pagamentos (Objetivo 4)

In [141]:
textPDF2 = textPDF.split('Envio de Nota de Empenho', 1)
if len(textPDF2) > 1:
  textPDF2 = textPDF2[1]
else:
  textPDF2 = ""


In [142]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')


# Supondo que 'text' é o conteúdo extraído do PDF
tokens = word_tokenize(textPDF2)
# Remover stopwords (palavras comuns que não têm significado relevante)
stop_words = set(stopwords.words('portuguese'))
textPDF2_tokenizado = [word for word in tokens if word.lower() not in stop_words]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [143]:
import re

lista_comprovantes = []

padrao1 = re.compile(r'^2024NE000\d{3}$')
padrao2 = re.compile(r'\d{2}[A-Z][a-z]{2}\d{2}')

for i in range (len(textPDF2_tokenizado)):
  if textPDF2_tokenizado[i] == '531814':
    comprovante = []
    for j in range (i+1, i+20):
      if padrao1.match(textPDF2_tokenizado[j]) or padrao2.match(textPDF2_tokenizado[j]):
        comprovante.append(textPDF2_tokenizado[j])
    lista_comprovantes.append(comprovante)

lista_comprovantes

lista_comprovantes_final = []
for sublist in lista_comprovantes:
  if sublist not in lista_comprovantes_final and len(sublist[1])<10:
    lista_comprovantes_final.append(sublist)


for lista in lista_comprovantes_final:
  lista[0] = lista[0][9:]

lista_comprovantes_final


[['333', '21Jun24'],
 ['334', '21Jun24'],
 ['332', '28Jun24'],
 ['336', '12Jul24'],
 ['335', '08Ago24'],
 ['335', '22Ago24']]

In [144]:
df_Notas_de_empenho = df_Notas_de_empenho.drop(columns=['NE_Completa'])

df_Notas_de_empenho['Data_Pag'] = 'Não'
df_Notas_de_empenho['NE'] = df_Notas_de_empenho['NE'].astype(str)

for i in range (len(lista_comprovantes_final)):
  for j in range (df_Notas_de_empenho.shape[0]):
    if lista_comprovantes_final[i][0] == df_Notas_de_empenho['NE'][j]:
      df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]

df_Notas_de_empenho

<ipython-input-144-ea290ebfdcf2>:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Notas_de_empenho['Data_Pag'][j] = lista_comprovantes_final[i][1]
<ipython-input-144-ea290ebfdcf2>:9: FutureWarning: ChainedAssignmentError: behaviour will c

,NE,CNPJ,Empresa,Valor,Email,itens_da_NE,Data_Pag
0,332,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"9.862,81",LCPARTICIPACOESCOMERCIO@GMAIL.COM,"[12, 13, 14, 15, 17, 18, 19, 21, 23, 24, 28, 3...",28Jun24
1,333,43.492.384/0001-10,ANA C REGIS,"311,14",anacregis.parnamirim@gmail.com,"[16, 2]",21Jun24
2,334,22.077.847/0001-07,JOSE DANTAS DINIZ FILHO,"381,75",dantasdiniz@gmail.com,[26],21Jun24
3,335,19.367.167/0001-13,COMERCIAL TARGET DE EPI E EPC LTDA,"1.592,50",licitacoes@targethospitalar.com.br,[46],22Ago24
4,336,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"6.491,60",doiskcomercio@gmail.com,"[3, 4, 5, 6, 7, 8, 11, 25, 27, 29, 33, 34, 48,...",12Jul24
5,326,43.492.384/0001-10,ANA C REGIS,"126,00",anacregis.parnamirim@gmail.com,[78],Não
6,348,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"380,00",doiskcomercio@gmail.com,"[1, 79, 30, 31]",Não
7,349,30.843.402/0001-19,DOIS K COMERCIO E SERVICOS LTDA,"145,50",doiskcomercio@gmail.com,"[9, 10, 57]",Não
8,350,43.492.384/0001-10,ANA C REGIS,"515,80",anacregis.parnamirim@gmail.com,[54],Não
9,351,54.416.857/0001-16,L & C PARTICIPACOES E COMERCIO LTDA,"124,95",LCPARTICIPACOESCOMERCIO@GMAIL.COM,[55],Não


In [146]:
df_Notas_de_empenho.to_excel('Planilha de Controle de Pagamentos.xlsx', index=False)

In [147]:
from IPython.display import display, HTML
# Texto em vermelho e com fonte maior
text1 = '<h1 style="color: red;">Objetivo 4 atingido</h1>'
text1 = '<h1 style="color: red;">Verifique se o arquivo "Planilha de Controle de Pagamentos.xlsx" foi criado.</h1>'

# Exibir o texto
display(HTML(text1))